In [18]:
# ============================================================
# CELL 1: IMPORT LIBRARIES
# ============================================================

print("=" * 60)
print("  IMPORTING LIBRARIES")
print("=" * 60)

# Core Libraries
import os
import cv2
import numpy as np
import pandas as pd

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.layers import (
    Dense, Input, Layer, GlobalAveragePooling1D,
    LayerNormalization, Lambda
)
from tensorflow.keras.models import Model

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Check GPU
print(f"\n✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

print("\n" + "=" * 60)
print("✓ LIBRARIES LOADED")
print("=" * 60)


  IMPORTING LIBRARIES

✅ TensorFlow version: 2.18.0
✅ GPU Available: False

✓ LIBRARIES LOADED


In [19]:
# ============================================================
# CELL 2: MULTI-HEAD ATTENTION LAYER
# ============================================================

class MultiHeadAttention(Layer):
    """Multi-Head Attention for spatial-temporal processing."""

    def __init__(self, num_heads, key_dim, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.d_model = num_heads * key_dim

        self.query_dense = Dense(self.d_model)
        self.key_dense = Dense(self.d_model)
        self.value_dense = Dense(self.d_model)
        self.combine_heads = Dense(self.d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.key_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, query, value, key):
        batch_size = tf.shape(query)[0]

        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        matmul_qk = tf.matmul(query, key, transpose_b=True)
        dk = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, value)

        output = tf.transpose(output, perm=[0, 2, 1, 3])
        output = tf.reshape(output, (batch_size, -1, self.d_model))

        return self.combine_heads(output)

    def get_config(self):
        config = super(MultiHeadAttention, self).get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
        })
        return config

print("✅ MultiHeadAttention layer defined!")


✅ MultiHeadAttention layer defined!


In [20]:
# ============================================================
# CELL 3: VISION TEMPORAL TRANSFORMER LAYER
# ============================================================

class VisionTemporalTransformer(Layer):
    """Vision Temporal Transformer for video processing."""

    def __init__(self, patch_size=8, d_model=128, num_heads=4,
                 spatial_layers=1, temporal_layers=1, **kwargs):
        super(VisionTemporalTransformer, self).__init__(**kwargs)
        self.patch_size = patch_size
        self.d_model = d_model
        self.num_heads = num_heads
        self.spatial_layers = spatial_layers
        self.temporal_layers = temporal_layers

        self.dense_projection = Dense(d_model)
        self.pos_emb = None

        # Spatial transformer
        self.spatial_mhas = [
            MultiHeadAttention(num_heads=num_heads, key_dim=d_model//num_heads)
            for _ in range(spatial_layers)
        ]
        self.spatial_norm1 = [LayerNormalization() for _ in range(spatial_layers)]
        self.spatial_ffn = [
            tf.keras.Sequential([
                Dense(d_model*4, activation='relu'),
                Dense(d_model)
            ]) for _ in range(spatial_layers)
        ]
        self.spatial_norm2 = [LayerNormalization() for _ in range(spatial_layers)]

        # Temporal transformer
        self.temporal_mhas = [
            MultiHeadAttention(num_heads=num_heads, key_dim=d_model//num_heads)
            for _ in range(temporal_layers)
        ]
        self.temporal_norm1 = [LayerNormalization() for _ in range(temporal_layers)]
        self.temporal_ffn = [
            tf.keras.Sequential([
                Dense(d_model*4, activation='relu'),
                Dense(d_model)
            ]) for _ in range(temporal_layers)
        ]
        self.temporal_norm2 = [LayerNormalization() for _ in range(temporal_layers)]

    def build(self, input_shape):
        H = input_shape[2]
        W = input_shape[3]
        ph = H // self.patch_size
        pw = W // self.patch_size
        num_patches = ph * pw

        self.pos_emb = self.add_weight(
            shape=(1, num_patches, self.d_model),
            initializer='random_normal',
            trainable=True,
            name='pos_emb'
        )
        super(VisionTemporalTransformer, self).build(input_shape)

    def call(self, inputs):
        input_shape = inputs.get_shape()
        shape = tf.shape(inputs)
        batch = shape[0]
        frames = shape[1]
        H = shape[2]
        W = shape[3]
        C_static = input_shape[-1]
        C = C_static if C_static is not None else shape[4]

        reshaped = tf.reshape(inputs, [-1, H, W, C])

        patches = tf.image.extract_patches(
            images=reshaped,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )

        if C_static is not None:
            patch_dim_static = self.patch_size * self.patch_size * C_static
        else:
            patch_dim_static = None

        patch_dim_dynamic = tf.shape(patches)[-1]
        final_patch_dim = patch_dim_static if patch_dim_static is not None else patch_dim_dynamic

        patches = tf.reshape(patches, [-1, tf.shape(patches)[1] * tf.shape(patches)[2], final_patch_dim])

        if patch_dim_static is not None:
            patches.set_shape([None, None, patch_dim_static])

        x = self.dense_projection(patches) + self.pos_emb

        # Spatial transformer
        for i in range(self.spatial_layers):
            attn = self.spatial_mhas[i](x, x, x)
            x = x + attn
            x = self.spatial_norm1[i](x)
            ff = self.spatial_ffn[i](x)
            x = x + ff
            x = self.spatial_norm2[i](x)

        # Temporal pooling
        x = tf.reshape(x, [batch, frames, -1, self.d_model])
        x = tf.reduce_mean(x, axis=2)
        x.set_shape([None, None, self.d_model])

        # Temporal transformer
        for i in range(self.temporal_layers):
            attn = self.temporal_mhas[i](x, x, x)
            x = x + attn
            x = self.temporal_norm1[i](x)
            ff = self.temporal_ffn[i](x)
            x = x + ff
            x = self.temporal_norm2[i](x)

        pooled = GlobalAveragePooling1D()(x)
        return pooled

    def get_config(self):
        config = super(VisionTemporalTransformer, self).get_config()
        config.update({
            "patch_size": self.patch_size,
            "d_model": self.d_model,
            "num_heads": self.num_heads,
            "spatial_layers": self.spatial_layers,
            "temporal_layers": self.temporal_layers,
        })
        return config

print("✅ VisionTemporalTransformer layer defined!")


✅ VisionTemporalTransformer layer defined!


In [22]:
# ============================================================
# CELL 4: LOAD MODEL
# ============================================================

print("=" * 60)
print("  LOADING MODEL")
print("=" * 60)

MODEL_PATH = '/kaggle/input/best-lipinc-model-h5/tensorflow2/default/1/best_lipinc_model.h5'

print(f"\n📂 Model path: {MODEL_PATH}")

# Check if file exists
if not os.path.exists(MODEL_PATH):
    print(f"\n❌ ERROR: Model not found!")
    print(f"   Path: {MODEL_PATH}")
    raise FileNotFoundError("Model file not found")

file_size_mb = os.path.getsize(MODEL_PATH) / (1024 * 1024)
print(f"✅ Model file found! ({file_size_mb:.2f} MB)")

# Rebuild architecture
print(f"\n🏗️  Rebuilding model architecture...")

frame_input = Input(shape=(8, 64, 144, 3), name='FrameInput')
residue_input = Input(shape=(7, 64, 144, 3), name='ResidueInput')

vt = VisionTemporalTransformer(
    patch_size=8, d_model=128, num_heads=4,
    spatial_layers=1, temporal_layers=1
)

frame_feat = vt(frame_input)
residue_feat = vt(residue_input)

expand1 = Lambda(lambda x: tf.expand_dims(x, axis=1), output_shape=(1, 128))
q = expand1(frame_feat)
k = expand1(residue_feat)
v = k

mha = MultiHeadAttention(num_heads=4, key_dim=32)
attn_out = mha(q, value=v, key=k)

squeeze = Lambda(lambda x: tf.squeeze(x, axis=1), output_shape=(128,))
attn_out = squeeze(attn_out)

concat = Lambda(lambda t: tf.concat(t, axis=1), output_shape=(384,))
fusion = concat([frame_feat, residue_feat, attn_out])

x = Dense(512, activation='relu')(fusion)
x = Dense(256, activation='relu')(x)
class_output = Dense(2, activation='softmax', name='class_output')(x)
features_output = Dense(128, activation=None, name='features_output')(x)

model = Model(
    inputs=[frame_input, residue_input],
    outputs=[class_output, features_output],
    name='LIPINC_Model'
)

print("✅ Architecture rebuilt!")

# Load weights
print(f"\n📥 Loading weights...")
model.load_weights(MODEL_PATH)
print("✅ Weights loaded successfully!")

print(f"\n📊 Model Info:")
print(f"   Total parameters: {model.count_params():,}")

print("\n" + "=" * 60)
print("✓ MODEL READY FOR INFERENCE")
print("=" * 60)


  LOADING MODEL

📂 Model path: /kaggle/input/best-lipinc-model-h5/tensorflow2/default/1/best_lipinc_model.h5
✅ Model file found! (10.06 MB)

🏗️  Rebuilding model architecture...
✅ Architecture rebuilt!

📥 Loading weights...
✅ Weights loaded successfully!

📊 Model Info:
   Total parameters: 867,586

✓ MODEL READY FOR INFERENCE


In [23]:
# ============================================================
# CELL 5: VIDEO PROCESSOR (SAME AS V6)
# ============================================================

class VideoProcessor:
    """Video frame extraction with transition detection (V6 method)."""

    def __init__(self, frame_count=8, dim=(64, 144), use_transitions=True):
        self.frame_count = frame_count
        self.dim = dim
        self.use_transitions = use_transitions

    def select_transition_frames(self, path):
        """Select frames at scene transitions."""
        cap = cv2.VideoCapture(path)
        prev_gray = None
        transition_scores = []
        all_frames = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if prev_gray is not None:
                diff = cv2.absdiff(gray, prev_gray)
                mean_diff = np.mean(diff)
                transition_scores.append(mean_diff)
            else:
                transition_scores.append(0)

            all_frames.append(frame)
            prev_gray = gray

        cap.release()

        if len(all_frames) >= self.frame_count:
            top_indices = np.argsort(transition_scores)[-self.frame_count:]
            top_indices = sorted(top_indices)
        else:
            top_indices = list(range(len(all_frames)))

        frames = []
        for idx in top_indices[:self.frame_count]:
            frame = all_frames[idx]
            frame = cv2.resize(frame, (self.dim[1], self.dim[0]))
            frames.append(frame)

        while len(frames) < self.frame_count:
            frames.append(np.zeros((self.dim[0], self.dim[1], 3)))

        return np.array(frames).astype(np.float32) / 255.0

    def load_video(self, path):
        """Load video with transition detection."""
        return self.select_transition_frames(path)

    def compute_residue(self, frames):
        """Compute frame differences."""
        residues = np.zeros((self.frame_count - 1, self.dim[0], self.dim[1], 3), dtype=np.float32)
        if len(frames) > 1:
            for i in range(1, len(frames)):
                residues[i-1] = frames[i] - frames[i-1]
        return residues

# Initialize processor
processor = VideoProcessor(frame_count=8, dim=(64, 144), use_transitions=True)

print("✅ VideoProcessor initialized!")
print("   Frame count: 8")
print("   Dimensions: (64, 144)")
print("   Method: Transition Detection (V6)")


✅ VideoProcessor initialized!
   Frame count: 8
   Dimensions: (64, 144)
   Method: Transition Detection (V6)


In [36]:
# ============================================================
# CELL 6: TEST SINGLE VIDEO (WITH VERIFICATION)
# ============================================================

print("=" * 60)
print("  TEST SINGLE VIDEO")
print("=" * 60)

# -------------------- VIDEO PATH --------------------
VIDEO_PATH = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_manipulated_sequences/DFD_manipulated_sequences/01_03__podium_speech_happy__480LQD1C.mp4"

# Configuration
CONFIDENCE_THRESHOLD = 0.75  # 75% threshold

if not os.path.exists(VIDEO_PATH):
    print(f"\n❌ Video not found: {VIDEO_PATH}")
else:
    video_name = os.path.basename(VIDEO_PATH)
    print(f"\n📹 Video: {video_name}")

    # -------------------- LABEL DETECTION --------------------
    path_lower = VIDEO_PATH.lower()

    # From filename
    if "fake" in video_name.lower():
        filename_label = "FAKE"
    elif "real" in video_name.lower():
        filename_label = "REAL"
    else:
        filename_label = "UNKNOWN"

    # From folder path
    if "/fake/" in path_lower:
        folder_label = "FAKE"
    elif "/real/" in path_lower:
        folder_label = "REAL"
    else:
        folder_label = "UNKNOWN"

    # Expected label (prioritize folder)
    if folder_label != "UNKNOWN":
        expected_label = folder_label
    elif filename_label != "UNKNOWN":
        expected_label = filename_label
    else:
        expected_label = None

    # Show detected labels
    if expected_label:
        print(f"   Expected: {expected_label}")

        # Check for conflicts
        if filename_label != "UNKNOWN" and folder_label != "UNKNOWN":
            if filename_label != folder_label:
                print(f"   ⚠️  WARNING: Filename says '{filename_label}' but folder is '{folder_label}'")

    # -------------------- PROCESS --------------------
    print(f"\n🔮 Processing...")

    try:
        # Load frames
        frames = processor.load_video(VIDEO_PATH)
        residues = processor.compute_residue(frames)

        # Prepare input
        frames_input = np.expand_dims(frames, axis=0)
        residues_input = np.expand_dims(residues, axis=0)

        # Predict
        predictions = model.predict([frames_input, residues_input], verbose=0)
        probs = predictions[0][0]

        prob_real = probs[0]
        prob_fake = probs[1]
        prediction = "REAL" if prob_real > prob_fake else "FAKE"
        confidence = max(prob_real, prob_fake)

        # -------------------- RESULTS --------------------
        print("\n" + "=" * 60)
        print("  RESULTS")
        print("=" * 60)

        print(f"\n📊 Probabilities:")
        print(f"   Real: {prob_real:.4f} ({prob_real*100:.2f}%)")
        print(f"   Fake: {prob_fake:.4f} ({prob_fake*100:.2f}%)")

        print(f"\n🎯 Prediction: {prediction}")
        print(f"   Confidence: {confidence*100:.2f}%")

        # Check correctness
        if expected_label:
            correct = (prediction == expected_label)
            print(f"\n   Expected: {expected_label}")
            print(f"   Result: {'✅ CORRECT' if correct else '❌ INCORRECT'}")

        # Confidence analysis
        print(f"\n📈 Confidence Level:")
        if confidence > 0.95:
            print("   🟢 Very High - Model is very certain")
        elif confidence > 0.85:
            print("   🟢 High - Model is confident")
        elif confidence > 0.75:
            print("   🟡 Medium - Acceptable confidence")
        elif confidence > 0.60:
            print("   🟠 Low - Model is uncertain")
        else:
            print("   🔴 Very Low - High uncertainty!")

        # Threshold check
        if confidence < CONFIDENCE_THRESHOLD:
            print(f"\n   ⚠️  WARNING: Confidence below threshold ({CONFIDENCE_THRESHOLD*100:.0f}%)")
            print(f"   Difference: {(prob_fake - prob_real)*100:.2f}%")
            print(f"   Consider manual review or additional testing")

        # Ambiguous case
        prob_diff = abs(prob_real - prob_fake)
        if prob_diff < 0.15:  # Less than 15% difference
            print(f"\n   ⚠️  AMBIGUOUS: Probabilities are too close!")
            print(f"   Difference: {prob_diff*100:.2f}%")
            print(f"   Model cannot clearly distinguish")

        print("\n" + "=" * 60)

    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        import traceback
        traceback.print_exc()


  TEST SINGLE VIDEO

📹 Video: 01_03__podium_speech_happy__480LQD1C.mp4

🔮 Processing...

  RESULTS

📊 Probabilities:
   Real: 0.2524 (25.24%)
   Fake: 0.7476 (74.76%)

🎯 Prediction: FAKE
   Confidence: 74.76%

📈 Confidence Level:
   🟠 Low - Model is uncertain

   ⚠️  WARNING: Confidence below threshold (75%)
   Difference: 49.53%
   Consider manual review or additional testing



In [42]:
# ============================================================
# CELL: CHECK TRAINING HISTORY
# ============================================================

import pandas as pd

HISTORY_PATH = '/content/drive/MyDrive/DeepFake Detection Project Material/Final Version/training_history.csv'

if os.path.exists(HISTORY_PATH):
    history_df = pd.read_csv(HISTORY_PATH)

    print("=" * 60)
    print("  TRAINING HISTORY")
    print("=" * 60)

    # Best epoch
    best_epoch = history_df['val_class_output_accuracy'].idxmax()

    train_acc = history_df['class_output_accuracy'].iloc[best_epoch]
    val_acc = history_df['val_class_output_accuracy'].iloc[best_epoch]

    print(f"\n📊 Best Epoch ({best_epoch + 1}):")
    print(f"   Train Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"   Val Accuracy:   {val_acc:.4f} ({val_acc*100:.2f}%)")
    print(f"   Gap: {(train_acc - val_acc):.4f} ({(train_acc - val_acc)*100:.2f}%)")

    # Show all epochs
    print(f"\n📈 Training Progress:")
    print(f"{'Epoch':<8} {'Train Acc':<12} {'Val Acc':<12} {'Gap':<10}")
    print("=" * 50)

    for i in range(len(history_df)):
        t_acc = history_df['class_output_accuracy'].iloc[i]
        v_acc = history_df['val_class_output_accuracy'].iloc[i]
        gap = t_acc - v_acc
        marker = " ← Best" if i == best_epoch else ""
        print(f"{i+1:<8} {t_acc:.4f}      {v_acc:.4f}      {gap:+.4f}{marker}")

else:
    print("❌ Training history not found!")
    print(f"   Looking for: {HISTORY_PATH}")


  TRAINING HISTORY

📊 Best Epoch (13):
   Train Accuracy: 0.8485 (84.85%)
   Val Accuracy:   0.8435 (84.35%)
   Gap: 0.0050 (0.50%)

📈 Training Progress:
Epoch    Train Acc    Val Acc      Gap       
1        0.5807      0.6919      -0.1112
2        0.6420      0.6968      -0.0548
3        0.7133      0.7677      -0.0544
4        0.7568      0.7628      -0.0060
5        0.7804      0.6748      +0.1056
6        0.7993      0.6553      +0.1440
7        0.8118      0.6822      +0.1297
8        0.8155      0.7482      +0.0673
9        0.8318      0.8362      -0.0044
10       0.8375      0.8289      +0.0087
11       0.8412      0.8264      +0.0148
12       0.8454      0.8337      +0.0116
13       0.8485      0.8435      +0.0050 ← Best
14       0.8532      0.8386      +0.0146
15       0.8595      0.8313      +0.0282
16       0.8574      0.8117      +0.0457
17       0.8643      0.8068      +0.0574
18       0.8643      0.7824      +0.0819
19       0.8684      0.8386      +0.0298
20       0.869